In [6]:
import os
import sys
import subprocess
import glob
import json
from pathlib import Path
import pandas
import pickle
import pprint
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastpCommandline
from axe import axe
from collections import defaultdict

In [7]:
RUN_BLAST = True

In [8]:
def parse_blast_results(blast_results : str) -> dict:
    file_size = os.path.getsize(blast_results)
    if file_size == 0:
        return None
    results = {}
    with open(blast_results, "r") as f:
        blast_records = NCBIXML.parse(f)
        for record in blast_records:
            results[record.query] = []
            for alignment in record.alignments:
                for hsp in alignment.hsps:
                    results[record.query].append({'contig_id' : record.query,
                                                  'contig_len' : record.query_length,
                                                  'hit_id' : alignment.hit_id,  # Modify this line to use alignment.hit_id instead of alignment.title
                                                  'hit_title' : alignment.title,
                                                  'hit_def' : alignment.hit_def,
                                                  'hit_expect' : hsp.expect,
                                                  'hit_score' : hsp.score,
                                                  'hit_query_start' : hsp.query_start,
                                                  'hit_query_end' : hsp.query_end,
                                                  'hit_subject_start' : hsp.sbjct_start,
                                                  'hit_subject_end' : hsp.sbjct_end,
                                                  'hit_idents' : hsp.identities,
                                                  'hit_pos' : hsp.positives,
                                                  'hit_gaps' : hsp.gaps,
                                                  'hit_frame' : hsp.frame,
                                                  'hit_query' : hsp.query,
                                                  'hit_subject' : hsp.sbjct})
    return results

In [9]:
mjf = axe.mjf(logfile_name='plasmid_identification_v3_parsing')

--- Logging error ---
Traceback (most recent call last):
  File "/home/mf019/miniconda3/lib/python3.12/logging/__init__.py", line 1160, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "/home/mf019/miniconda3/lib/python3.12/logging/__init__.py", line 999, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "/home/mf019/miniconda3/lib/python3.12/site-packages/axe/axe.py", line 74, in format
    return formatter.format(record)
           ^^^^^^^^^^^^^^^^
AttributeError: 'tuple' object has no attribute 'format'
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/mf019/miniconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/mf019/miniconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/mf019/miniconda3/lib/python3.

In [10]:
testing_set_file = 'dbs/Minimal-PF32-testor-set-March2021.faa'


In [28]:
# get working directory
init_cwd = os.getcwd()
# Set working directory
working_dir = '/home/mf019/borrelia_plasmid_classifier_v3'
# path out the databases
databases_dir = f'{working_dir}/dbs'
whole_plasmid_db = f'{databases_dir}/plasmid_db/plasmid_db'
pf32_db = f'{databases_dir}/pf32_db/pf32'
# lets specify where parsing tables are to be found!
parsing_tables_dir = f'{working_dir}/parsing_tables'
# set assemblies directory # DO I WANT TO ARGV THIS?
assemblies_dir = '/home/mf019/longread_GWAS/longread_analysis/paired_assemblies/paired_only' #f'{working_dir}/assemblies'
shortread_dir  = f'{assemblies_dir}/shortread'
longread_dir = f'{assemblies_dir}/longread'
shortread_contigs  = f'{shortread_dir}/contigs'
longread_contigs = f'{longread_dir}/contigs'
shortread_annotations  = f'{shortread_dir}/annotation'
longread_annotations = f'{longread_dir}/annotation'
output_dir = f'{working_dir}/output'
blast_results_dir = f'{output_dir}/blast_results'
pf32_blast_results = f'{blast_results_dir}/pf32'
plasmid_blast_results = f'{blast_results_dir}/whole_plasmid'

In [29]:

print(f'Working directory: {working_dir}')
print(f'Databases directory: {databases_dir}')
print(f'Plasmid database: {whole_plasmid_db}')
print(f'PF32 database: {pf32_db}')
print(f'Parsing tables directory: {parsing_tables_dir}')
print(f'Assemblies directory: {assemblies_dir}')
print(f'Shortread directory: {shortread_dir}')
print(f'Longread directory: {longread_dir}')
print(f'Shortread contigs directory: {shortread_contigs}')
print(f'Shortread annotations directory: {shortread_annotations}')
print(f'Longread contigs directory: {longread_contigs}')
print(f'Longread annotations directory: {longread_annotations}')
print(f'Output directory: {output_dir}')
print(f'Whole contig blast results directory: {blast_results_dir}')
print(f'PF32 Blast results: {pf32_blast_results}')



Working directory: /home/mf019/borrelia_plasmid_classifier_v3
Databases directory: /home/mf019/borrelia_plasmid_classifier_v3/dbs
Plasmid database: /home/mf019/borrelia_plasmid_classifier_v3/dbs/plasmid_db/plasmid_db
PF32 database: /home/mf019/borrelia_plasmid_classifier_v3/dbs/pf32_db/pf32
Parsing tables directory: /home/mf019/borrelia_plasmid_classifier_v3/parsing_tables
Assemblies directory: /home/mf019/longread_GWAS/longread_analysis/paired_assemblies/paired_only
Shortread directory: /home/mf019/longread_GWAS/longread_analysis/paired_assemblies/paired_only/shortread
Longread directory: /home/mf019/longread_GWAS/longread_analysis/paired_assemblies/paired_only/longread
Shortread contigs directory: /home/mf019/longread_GWAS/longread_analysis/paired_assemblies/paired_only/shortread/contigs
Shortread annotations directory: /home/mf019/longread_GWAS/longread_analysis/paired_assemblies/paired_only/shortread/annotation
Longread contigs directory: /home/mf019/longread_GWAS/longread_analysis

In [30]:
# get annotations for each method
sr_gbs = glob.glob(f'{shortread_annotations}/*/*.gbff')
print(f'Found {len(sr_gbs)} shortread gbff files')
lr_gbs = glob.glob(f'{longread_annotations}/*/*.gbff')
print(f'Found {len(lr_gbs)} longread gbff files')

Found 49 shortread gbff files
Found 49 longread gbff files


In [31]:

if not os.path.exists(f'{output_dir}/assembly_dict.pkl'):
    assembly_dict = defaultdict(dict)
    for assembly in lr_gbs: # iterate through each longread assembly
        assembly_name = Path(assembly).stem # get the name of the assembly
        print(f'Processing assembly: {assembly_name}')
        if assembly_name.endswith(('H','P')):
            assembly_name = assembly_name[:-1] # chop last character off for dict key
        assembly_dict[assembly_name] = defaultdict(dict)
        assembly_dict[assembly_name]['longread'] = defaultdict(dict)
        with open(assembly, 'r') as f: # get longread contigs and annotations
            num_contigs = 0
            num_genes = 0
            for contig in SeqIO.parse(handle=f, format='genbank'):
                num_contigs += 1
                contig_id = contig.name
                print(contig_id)
                assembly_dict[assembly_name]['longread'][contig_id]['seq'] = contig.seq
                assembly_dict[assembly_name]['longread'][contig_id]['annotations'] = contig.annotations
                assembly_dict[assembly_name]['longread'][contig_id]['features'] = contig.features
                if len(contig.features) > 0:
                    for feature in contig.features:
                        if feature.type == 'CDS':
                            num_genes += 1
            print(f'longread:{assembly_name}[\'longread\'] has {num_contigs} contigs')
            print(f'longread: {assembly_name}[\'longread\'] has {num_genes} annotated genes')

        # now we need to get the shortread contigs and annotations if they exist.
        shortread_file = f'{shortread_annotations}/{assembly_name}.gbff'
        if shortread_file in sr_gbs:
            print(f'Found shortread annotations for {assembly_name}')
            assembly_dict[assembly_name]['shortread'] = defaultdict(dict)
            with open(shortread_file, 'r') as fi:
                num_genes = 0
                num_contigs = 0
                for scontig in SeqIO.parse(handle=fi, format='genbank'):
                    num_contigs += 1
                    scontig_id = scontig.id
                    #print(scontig_id)
                    assembly_dict[assembly_name]['shortread'][scontig_id]['seq'] = scontig.seq
                    assembly_dict[assembly_name]['shortread'][scontig_id]['annotations'] = scontig.annotations
                    assembly_dict[assembly_name]['shortread'][scontig_id]['features'] = scontig.features
                    if len(scontig.features) > 0:
                        for feature in scontig.features:
                            if feature.type == 'CDS':
                                num_genes += 1
                    # now we need to blast this contig against the pf32 and whole plasmid databases
                    # we will need to save these results in a dictionary
            print(f'shortread:{assembly_name} has {num_contigs} contigs')
            print(f'shortread: {assembly_name} has {num_genes} annotated genes')
        else:
            assembly_dict[assembly_name]['shortread'] = 'NOT AVAILABLE'
            print(f'No shortread annotations found for {assembly_name}')
    print('dictionary construction complete! pickling!')
    pickle.dump(assembly_dict, open(f'{output_dir}/assembly_dict_v3.pkl', 'wb'))
    print(f'dictionary pickled to: {output_dir}/assembly_dict_v3.pkl')


Processing assembly: URI87H
contig000001
contig000002
contig000003
contig000004
contig000005
contig000006
contig000007
contig000008
contig000009
contig000010
contig000011
contig000012
contig000013
contig000014
contig000015
contig000016
contig000017
contig000018
contig000019
contig000020
contig000021
contig000022
contig000023
contig000024
contig000025
contig000026
contig000027
contig000028
contig000029
contig000030
contig000031
contig000032
contig000033
contig000034
contig000035
contig000036
contig000037
contig000038
contig000039
contig000040
contig000041
contig000042
contig000043
contig000044
contig000045
contig000046
contig000047
contig000048
contig000049
contig000050
contig000051
contig000052
contig000053
contig000054
contig000055
contig000056
contig000057
contig000058
contig000059
contig000060
contig000061
contig000062
contig000063
contig000064
contig000065
contig000066
longread:URI87['longread'] has 66 contigs
longread: URI87['longread'] has 1466 annotated genes
No shortread annota

In [39]:
if RUN_BLAST is True:
    for isolate in assembly_dict:
        input_files = [] # empty list to store input files
        lr_data = assembly_dict[isolate]['longread']
        sr_data = assembly_dict[isolate]['shortread']
        lr_id = isolate # since we're operating on lr first
        input_files.append(f'{longread_contigs}/{lr_id}.fasta') # add lr contigs to list
        if sr_data == 'NOT AVAILABLE':
            print(f'No shortread annotations found for {isolate}')
            sr_id = None
        else:
            sr_id = isolate[:-1:]
            input_files.append(f'{shortread_contigs}/{sr_id}.fasta')
        print(f'setting up blast commands for {isolate}')
        for file in input_files:
            name = Path(file).stem
            path = os.path.dirname(file)
            print(path)
            bind_path = f'/tmp/{name}.fasta'
            wp_output_file = f'{plasmid_blast_results}/{name}_whole_plasmid.xml'
            wp_blast_cmd = f'blastn -query {bind_path} -task "blastn" '
            wp_blast_cmd += f'-db {whole_plasmid_db} '
            wp_blast_cmd += f' -out {wp_output_file} '
            wp_blast_cmd += "-evalue 1e-100 -num_threads 60 -outfmt 5 -max_target_seqs 5 -max_hsps 5"
            print(f'Running blast command:\n{wp_blast_cmd}')
            #subprocess.run(f'singularity exec --bind {path}:/tmp containers/blast.simg {wp_blast_cmd}', shell=True, check=False) # run it
            print(f'whole plasmid blast results written to: {wp_output_file}')
            print(f'setting up blast command for pf32 blast')
            pf_output_file = f'{pf32_blast_results}/{name}_pf32.xml'
            pf_blast_cmd = f'blastx -query {bind_path} -task "blastx" '
            pf_blast_cmd += f'-db {pf32_db} '
            pf_blast_cmd += f' -out {pf_output_file} '
            pf_blast_cmd += "-evalue 1e-100 -num_threads 60 -outfmt 5 -max_target_seqs 5 -max_hsps 5"
            print(f'Running blast command:\n{pf_blast_cmd}')
            #subprocess.run(f'singularity exec --bind {path}:/tmp containers/blast.simg {pf_blast_cmd}', shell=True, check=False) # run it, but don't check for errors to avoid stopping the script
            print(f'pf32 blast results written to: {pf_output_file}')
    print('saddle up, we are done here!')
else:
    print('Blast already run, skipping blast')
    

No shortread annotations found for URI87
setting up blast commands for URI87
/home/mf019/longread_GWAS/longread_analysis/paired_assemblies/paired_only/longread/contigs
Running blast command:
blastn -query /tmp/URI87.fasta -task "blastn" -db /home/mf019/borrelia_plasmid_classifier_v3/dbs/plasmid_db/plasmid_db  -out /home/mf019/borrelia_plasmid_classifier_v3/output/blast_results/whole_plasmid/URI87_whole_plasmid.xml -evalue 1e-100 -num_threads 60 -outfmt 5 -max_target_seqs 5 -max_hsps 5
whole plasmid blast results written to: /home/mf019/borrelia_plasmid_classifier_v3/output/blast_results/whole_plasmid/URI87_whole_plasmid.xml
setting up blast command for pf32 blast
Running blast command:
blastx -query /tmp/URI87.fasta -task "blastx" -db /home/mf019/borrelia_plasmid_classifier_v3/dbs/pf32_db/pf32  -out /home/mf019/borrelia_plasmid_classifier_v3/output/blast_results/pf32/URI87_pf32.xml -evalue 1e-100 -num_threads 60 -outfmt 5 -max_target_seqs 5 -max_hsps 5
pf32 blast results written to: /h

## ~~Ok so now let's strip out the pfam32s and blast those against the testor set from Ira!~~ will implement later after parsing 

In [ ]:
# let's define some possible annotated names!
#genbanks = glob.glob(f'{assembly_annotations}/*.gbff')

In [ ]:
# lets make a dictionary containing our contigs, parse the genbank and pull the top level details for this.
# I forgot that I already did this below but this can hang around in case I need it (I probably will...)
#assembly_dict = {}
#for file in genbanks:
#    print(file.split('/')[-1])
#    sample_id = file.split("/")[-1].split(".")[0]
#    assembly_dict[sample_id] = {}
#    with open(file, "r") as f:
#        record = SeqIO.parse(f, "genbank")
#        for rec in record:
#            contig_id = rec.id
#            assembly_dict[sample_id]["contigs"] = {contig_id: {}}
#            contig_sequence = rec.seq
#            contig_len = len(rec.seq)
#            num_feats = len(rec.features)
#            assembly_dict[sample_id]["contigs"][contig_id] = {"contig_seq" : contig_sequence, "contig_len": contig_len, "feature_count": num_feats}

In [ ]:
#pfam32_names = ["parA",'ParA','pf32','partition'] # this needs considerable WORK, # I may just blast one of the testors and see what possible names come back.
#parA_genes = {}
#for file in genbanks:
#    print(file)
#    sample_id = file.split("/")[-1].split(".")[0]
#    parA_genes[sample_id] = []
#    with open(file, "r") as f:
#        record = SeqIO.parse(f, "genbank")
#        for rec in record:
#            #print(rec.id)
#            for feature in rec.features:
#                if feature.type == "CDS":
#                    product = feature.qualifiers['product']
#                    if any(pfam32_names) in product[0]:
#                        feature.qualifiers['product']
#                        nucseq = feature.extract(rec.seq)
#                        if len(nucseq) % 3 != 0:
#                            print(f"Sequence length not divisible by 3 for {rec.id}---{sample_id}")
#                            nucseq += "N"
#                            print("adding N!")
#                        else:
#                            protseq = nucseq.translate(table=11, to_stop=True)
#                            gene = SeqIO.SeqRecord(protseq, id=f"{sample_id}-{rec.id}-ParA", name=product[0], description=f"{rec.id}---{sample_id}", dbxrefs=None)
#                            parA_genes[sample_id].append(gene)

In [ ]:
## now write out to file for blastin!
#with open('parA_genes_query_v2.faa', 'w') as out:
#    for sample in parA_genes:
#        for gene in parA_genes[sample]:
#            print(gene)
#            SeqIO.write(gene, out, "fasta")

## Let's parse our blast hits now!

In [ ]:
pf32_results = glob.glob(f'{pf32_blast_results}/*.xml')
wp_results = glob.glob(f'{plasmid_blast_results}/*.xml')
mjf.logdebug(f'Found {len(pf32_results)} pf32 blast results')
mjf.logdebug(f'Found {len(wp_results)} whole plasmid blast results')

In [ ]:
for result in pf32_results:
    print(result)
    name = Path(result).stem.split("_")[0]
    if name.endswith(('H','P')):
        print(f'input is longread: {name}')
        name = name[:-1:] # chop the char for dict lookup.
        current_asm = assembly_dict[name]['longread']
    else:
        print(f'input is shortread: {name}')
        current_asm = assembly_dict[name]['shortread']
    pf32_hits = parse_blast_results(result)
    for hit in pf32_hits:
        for k, v in enumerate(pf32_hits[hit]):
            curr_contig = current_asm[v["contig_id"]]
            curr_contig['pf32_hits'] = v

In [ ]:
for result in wp_results:
    print(result)
    name = Path(result).stem.split("_")[0]
    if name.endswith(('H','P')):
        print(f'input is longread: {name}')
        name = name[:-1:] # chop the char for dict lookup.
        current_asm = assembly_dict[name]['longread']
    else:
        print(f'input is shortread: {name}')
        current_asm = assembly_dict[name]['shortread']
    wp_hits = parse_blast_results(result)
    for hit in wp_hits:
        for k, v in enumerate(wp_hits[hit]):
            curr_contig = current_asm[v["contig_id"]]
            curr_contig['wp_hits'] = v
            #print(f'Contig: {v["contig_id"]}')
            #print(f'Contig Length: {v["contig_len"]}')
            #print(f'Hit: {v["hit_title"]}')
            #print(f'Hit_Def: {v["hit_def"]}')
            #print(f'Expect: {v["hit_expect"]}')
            #print(f'Score: {v["hit_score"]}')
            #print(f'Query Start: {v["hit_query_start"]}')
            #print(f'Query End: {v["hit_query_end"]}')
            #print(f'Subject Start: {v["hit_subject_start"]}')
            #print(f'Subject End: {v["hit_subject_end"]}')
            #print(f'Identities: {v["hit_idents"]}')
            #print(f'Positives: {v["hit_pos"]}')
            #print(f'Gaps: {v["hit_gaps"]}')
            #print(f'Frame: {v["hit_frame"]}')
            #print(f'Query:   {v["hit_query"]}')
            #print(f'Subject: {v["hit_subject"]}')

In [ ]:
pickle.dump(assembly_dict, open(f'{output_dir}/assembly_dict_v3_parsed.pkl', 'wb'))

In [ ]:
for isolate in assembly_dict:
    for method in assembly_dict[isolate]:
        for contig in assembly_dict[isolate][method]:
            print(f'Isolate: {isolate}')
            print(f'Method: {method}')
            print(f'Contig: {assembly_dict[isolate][method][contig].items()}')
            print(f'pf32_hits: {[hit["contig_id"] for hit in assembly_dict[isolate][method][contig]["pf32_hits"]]}')
            print(f'wp_hits: {[hit["contig_id"] for hit in assembly_dict[isolate][method][contig]["wp_hits"]]}')
